In [5]:
import torch
import os
from torchvision.datasets import CIFAR10
from linear_classifier import LinearClassifier
import matplotlib.pyplot as plt

In [7]:
# Dataset Download
download = not os.path.isdir("cifar-10-batches-py")
dset_train = CIFAR10(root=".", download=download, train=True)
dset_test = CIFAR10(root=".", train=False)

X = torch.tensor(dset_train.data, dtype=torch.float32).permute(0, 3, 1, 2).div_(255)
y = torch.tensor(dset_train.targets, dtype=torch.int64)

X.cuda()
y.cuda()

X_test = torch.tensor(dset_test.data, dtype=torch.float32).permute(0, 3, 1, 2).div_(255)
y_test = torch.tensor(dset_test.targets, dtype=torch.int64)

X_test.cuda()
y_test.cuda();

TypeError: __init__() missing 1 required positional argument: 'root'

In [ ]:
# Dataset Preprocessing

# Flatting
X = X.reshape(X.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Adding Bias
ones_train = torch.ones(X.shape[0], 1, device=X.device)
X = torch.cat([X, ones_train], dim=1)

ones_train = torch.ones(X_test.shape[0], 1, device=X_test.device)
X_test = torch.cat([X_test, ones_train], dim=1)

# Validation and Training Split
traning_ratio = 0.70
num_training = int(X.shape[0] * (1.0 - traning_ratio))
num_validation = X.shape[0] - num_training
X_train = X[num_training : num_training + num_validation]
y_train = y[num_training : num_training + num_validation]

X_val = X[0:num_training]
y_val = y[0:num_training]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  torch.Size([35000, 3073])
Train labels shape:  torch.Size([35000])
Validation data shape:  torch.Size([15000, 3073])
Validation labels shape:  torch.Size([15000])
Test data shape:  torch.Size([10000, 3073])
Test labels shape:  torch.Size([10000])


In [ ]:
# Classify

classifier = LinearClassifier()
learning_rate = 1e-2
regularization = 1e-2
number_of_iteration = 2000
number_of_item_in_batch = 200
history = classifier.train(X_train, y_train, X_val, y_val, learning_rate,
                           regularization, number_of_iteration, number_of_item_in_batch)

y_pred = classifier.predict(X_test)

accuracy = classifier.calculate_accuracy(y_test, y_pred)

print(f'Final Accuracy on Test: {accuracy*100:.2f}%')


Trainning Accuracy: 0.34, Validation Accuracy: 0.33
Final Accuracy on Test: 33.00%
